numpy==2.1.3
opencv_python==4.10.0.84
python== 3.12.7

Se ruleaza tot codul si se obtin rezultatele pentru primele doua task-uri. Pentru a testa este necesar sa se modifice antrenare_dir cu numele directorului cu datele de testare.

-base_dir este directorul curent unde se dezarhiveaza fisierul.
-antrenare_dir este directorul pentru imaginile de testare, se poate schimba cu numele folderului(spre exemplu 'evaluare\\fake_test')
-imagini_auxiliare_dir este directorul pentru imaginile auxiliare, stiind ca in directorul curent avem deja acest date, nu le-am mai incarcat Folosesc prima imagine din directorul imagini_auxiliare pentru a porni jocul si a face comparatia mutarilor
-rezultate_dir este directorul in care se vor pune rezultatele in urma rularii codului

In [21]:
import cv2 as cv
import numpy as np
import os

In [22]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [23]:
def extrage_careu(image):
    image_g = cv.GaussianBlur(image, (3, 3), 0)  
    image_hsv = cv.cvtColor(image_g, cv.COLOR_BGR2HSV)
    
    #detectarea pieselor galbene 3x din colturile tablei de joc
    l = np.array([20, 150, 150])  
    u = np.array([30, 255, 255])  
    
    mask_yellow = cv.inRange(image_hsv, l, u)

    kernel = np.ones((3, 3), np.uint8)
    mask_cleaned = cv.erode(mask_yellow, kernel)

    contours, _ = cv.findContours(mask_cleaned, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    top_left, top_right, bottom_left, bottom_right = None, None, None, None

    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)

        piece_top_left = (x, y)
        piece_top_right = (x + w, y)
        piece_bottom_left = (x, y + h)
        piece_bottom_right = (x + w, y + h)

        if top_left is None or piece_top_left[0] + piece_top_left[1] < top_left[0] + top_left[1]:
            top_left = piece_top_left
        if top_right is None or piece_top_right[0] - piece_top_right[1] > top_right[0] - top_right[1]:
            top_right = piece_top_right
        if bottom_left is None or piece_bottom_left[0] - piece_bottom_left[1] < bottom_left[0] - bottom_left[1]:
            bottom_left = piece_bottom_left
        if bottom_right is None or piece_bottom_right[0] + piece_bottom_right[1] > bottom_right[0] + bottom_right[1]:
            bottom_right = piece_bottom_right

    image_copy = image.copy()
    if top_left: cv.circle(image_copy, top_left, 10, (0, 255, 0), -1)
    if top_right: cv.circle(image_copy, top_right, 10, (0, 255, 0), -1)
    if bottom_left: cv.circle(image_copy, bottom_left, 10, (0, 255, 0), -1)
    if bottom_right: cv.circle(image_copy, bottom_right, 10, (0, 255, 0), -1)
    #show_image("Colțuri detectate", image_copy)

    puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
    width, height = 1960, 1960
    destination = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype='float32')

    M = cv.getPerspectiveTransform(puzzle, destination)
    result = cv.warpPerspective(image, M, (width, height))

    return result


In [24]:
lines_horizontal=[]
for i in range(0,1961,140):
    l=[]
    l.append((0,i))
    l.append((1959,i))
    lines_horizontal.append(l)

In [25]:
lines_vertical=[]
for i in range(0,1961,140):
    l=[]
    l.append((i,0))
    l.append((i,1959))
    lines_vertical.append(l)

In [26]:
pozitii_3x = [(0, 0), (0, 6), (0, 7), (0, 13), (6, 0), (6, 13), (7, 0), (7, 13), (13, 0), (13, 6), (13, 7), (13, 13)]
pozitii_2x = [
    (1, 1), (1, 12), (2, 2), (2, 11), (3, 3), (3, 10), (4, 4), (4, 9),
    (9, 4), (9, 9), (10, 3), (10, 10), (11, 2), (11, 11), (12, 1), (12, 12)
]
pozitii_impartire = [(1, 4), (1, 9), (4, 1), (4, 12), (9, 1), (9, 12), (12, 4), (12, 9)]
pozitii_scadere = [(2, 5), (2, 8), (5, 2), (5, 11), (8, 2), (8, 11), (11, 5), (11, 8)]
pozitii_adunare = [(3, 6), (4, 7), (6, 4), (7, 3), (6, 10), (7, 9), (9, 6), (10, 7)]
pozitii_inmultire = [(3, 7), (4, 6), (6, 3), (7, 4), (6, 9), (7, 10), (9, 7), (10, 6)]

In [27]:
def determina_piese_posibile(tabla, i,j, dict_piese):
    piese_posibile = []
    puncte = None
    #stanga
    if j != 0 and j != 1:
        if tabla[i][j-1] and tabla[i][j-2]:
            if (i,j) in pozitii_impartire:
                if int(tabla[i][j-1]) % int(tabla[i][j-2]) == 0 and int(tabla[i][j-2]) != 0:
                    puncte = tabla[i][j-1] // tabla[i][j-2]
                elif int(tabla[i][j-2]) % int(tabla[i][j-1]) == 0 and int(tabla[i][j-1]) != 0:
                    puncte = int(tabla[i][j-2]) // int(tabla[i][j-1])
                if puncte in dict_piese and dict_piese[puncte] > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_scadere:
                puncte = abs(int(tabla[i][j-1]) - int(tabla[i][j-2]))
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_adunare:
                puncte = int(tabla[i][j-1]) + int(tabla[i][j-2])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_inmultire:
                puncte = int(tabla[i][j-1]) * int(tabla[i][j-2])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            else:
                if int(tabla[i][j-1]) % int(tabla[i][j-2]) == 0 and int(tabla[i][j-2]) != 0:
                    puncte = int(tabla[i][j-1]) // int(tabla[i][j-2])
                elif int(tabla[i][j-2]) % int(tabla[i][j-1]) == 0 and int(tabla[i][j-1]) != 0:
                    puncte = int(tabla[i][j-2]) // int(tabla[i][j-1])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = abs(int(tabla[i][j-1]) - int(tabla[i][j-2]))
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = int(tabla[i][j-1]) + int(tabla[i][j-2])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = int(tabla[i][j-1]) * int(tabla[i][j-2])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
    #dreapta
    if j != 13 and j != 12:
        if tabla[i][j+1] and tabla[i][j+2]:
            if (i,j) in pozitii_impartire:
                if int(tabla[i][j+1]) % int(tabla[i][j+2]) == 0 and int(tabla[i][j+2]) != 0:
                    puncte = int(tabla[i][j+1]) // int(tabla[i][j+2])
                elif int(tabla[i][j+2]) % int(tabla[i][j+1]) == 0 and int(tabla[i][j+1]) != 0:
                    puncte = int(tabla[i][j+2]) // int(tabla[i][j+1])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_scadere:
                puncte = abs(int(tabla[i][j+1]) - int(tabla[i][j+2]))
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_adunare:
                puncte = int(tabla[i][j+1]) + int(tabla[i][j+2])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_inmultire:
                puncte = int(tabla[i][j+1]) * int(tabla[i][j+2])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            else:
                if int(tabla[i][j+1]) % int(tabla[i][j+2]) == 0 and int(tabla[i][j+2]) != 0:
                    puncte = int(tabla[i][j+1]) // int(tabla[i][j+2])
                elif int(tabla[i][j+2]) % int(tabla[i][j+1]) == 0 and int(tabla[i][j+1]) != 0:
                    puncte = int(tabla[i][j+2]) // int(tabla[i][j+1])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = abs(int(tabla[i][j+1]) - int(tabla[i][j+2]))
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = int(tabla[i][j+1]) + int(tabla[i][j+2])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = int(tabla[i][j+1]) * int(tabla[i][j+2])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
    #sus
    if i != 0 and i != 1:
        if tabla[i-1][j] and tabla[i-2][j]:
            if (i,j) in pozitii_impartire:
                if int(tabla[i-1][j]) % int(tabla[i-2][j]) == 0 and int(tabla[i-2][j]) != 0:
                    puncte = int(tabla[i-1][j]) // int(tabla[i-2][j])
                elif int(tabla[i-2][j]) % int(tabla[i-1][j]) == 0 and int(tabla[i-1][j]) != 0:
                    puncte = int(tabla[i-2][j]) // int(tabla[i-1][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_scadere:
                puncte = abs(int(tabla[i-1][j]) - int(tabla[i-2][j]))
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_adunare:
                puncte = int(tabla[i-1][j]) + int(tabla[i-2][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_inmultire:
                puncte = int(tabla[i-1][j]) * int(tabla[i-2][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            else:
                if int(tabla[i-1][j]) % int(tabla[i-2][j]) == 0 and int(tabla[i-2][j]) != 0:
                    puncte = int(tabla[i-1][j]) // int(tabla[i-2][j])
                elif tabla[i-2][j] % tabla[i-1][j] == 0 and int(tabla[i-1][j]) != 0:
                    puncte = int(tabla[i-2][j]) // int(tabla[i-1][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = abs(int(tabla[i-1][j]) - int(tabla[i-2][j]))
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = int(tabla[i-1][j]) + int(tabla[i-2][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = int(tabla[i-1][j]) * int(tabla[i-2][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
    #jos
    if i != 13 and i != 12:
        if tabla[i+1][j] and tabla[i+2][j]:
            if (i,j) in pozitii_impartire:
                if int(tabla[i+1][j]) % int(tabla[i+2][j]) == 0 and int(tabla[i+2][j]) != 0:
                    puncte = int(tabla[i+1][j]) // int(tabla[i+2][j])
                elif int(tabla[i+2][j]) % int(tabla[i+1][j]) == 0 and int(tabla[i+1][j]) != 0:
                    puncte = int(tabla[i+2][j]) // int(tabla[i+1][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_scadere:
                puncte = abs(int(tabla[i+1][j]) - int(tabla[i+2][j]))
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_adunare:
                puncte = int(tabla[i+1][j]) + int(tabla[i+2][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            elif (i,j) in pozitii_inmultire:
                puncte = int(tabla[i+1][j]) * int(tabla[i+2][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
            else:
                if int(tabla[i+1][j]) % int(tabla[i+2][j]) == 0 and int(tabla[i+2][j]) != 0:
                    puncte = int(tabla[i+1][j]) // int(tabla[i+2][j])
                elif int(tabla[i+2][j]) % int(tabla[i+1][j]) == 0 and int(tabla[i+1][j]) != 0:
                    puncte = int(tabla[i+2][j]) // int(tabla[i+1][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = abs(int(tabla[i+1][j]) - int(tabla[i+2][j]))
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = int(tabla[i+1][j]) + int(tabla[i+2][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
                puncte = int(tabla[i+1][j]) * int(tabla[i+2][j])
                if puncte in dict_piese and dict_piese.get(puncte) > 0:
                    piese_posibile.append(puncte)
    return piese_posibile


In [28]:
def determina_pozitie_piesa(mask1, mask2, lines_horizontal, lines_vertical):
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0] + 5 
            y_max = lines_vertical[j + 1][1][0] - 5
            x_min = lines_horizontal[i][0][1] + 5
            x_max = lines_horizontal[i + 1][1][1] - 5
            patch1 = mask1[x_min:x_max, y_min:y_max].copy()
            patch2 = mask2[x_min:x_max, y_min:y_max].copy()
            medie1 = np.mean(patch1)
            medie2 = np.mean(patch2)
            if abs(medie1-medie2)>100:
                # dif = abs(medie1-medie2)
                # print(dif)
                # show_image('patch2', patch2)
                return i,j

In [29]:
def determina_piesa(img, i, j, piese_posibile, lines_horizontal, lines_vertical):
    y_min = lines_vertical[j][0][0]
    y_max = lines_vertical[j + 1][1][0] 
    x_min = lines_horizontal[i][0][1] 
    x_max = lines_horizontal[i + 1][1][1] 
    patch = img[x_min:x_max, y_min:y_max].copy()

    patch = cv.resize(cv.cvtColor(patch, cv.COLOR_BGR2GRAY), (140, 140))

    maxi = -np.inf
    poz = -1
    
    template_dirs = [
        'templates8\\',
        'templates7\\',
        'templates6\\'
    ]
    
    for piesa in piese_posibile:
        for template_dir in template_dirs:
            template_path = os.path.join(template_dir, f'{piesa}.jpg')
            if os.path.exists(template_path):
                img_template = cv.imread(template_path, cv.IMREAD_GRAYSCALE)
                for scale_factor in [0.8, 0.9, 1]:
                    resized_template = cv.resize(img_template, (int(140 * scale_factor), int(140 * scale_factor)))
                    corr = cv.matchTemplate(patch, resized_template, cv.TM_CCOEFF_NORMED)
                    corr_max = np.max(corr)
                    if corr_max > maxi:
                        maxi = corr_max
                        poz = piesa
    if poz == -1:
        files = os.listdir('templates5')
        for file in files:
            img_template = cv.imread(os.path.join('templates5', file), cv.IMREAD_GRAYSCALE)
            img_template = cv.resize(img_template, (140, 140))
            corr = cv.matchTemplate(patch, img_template, cv.TM_CCOEFF_NORMED)
            corr_max = np.max(corr)
            if corr_max > maxi:
                maxi = corr_max
                poz = file.split('.')[0]
    return int(poz)


In [30]:
base_dir = "."
antrenare_dir = os.path.join(base_dir, "testare") #directorul cu imaginile de test
imagini_auxiliare_dir = os.path.join(base_dir, "imagini_auxiliare")
rezultate_dir = os.path.join(base_dir, "evaluare\\fisiere_solutie\\341_Cochiorca_Oana_Maria") #directorul cu fisierele de iesire
if not os.path.exists(rezultate_dir):
    os.makedirs(rezultate_dir)
files = os.listdir(antrenare_dir)
img1_path = os.path.join(imagini_auxiliare_dir, "01.jpg")
img1 = cv.imread(img1_path)

tabla_joc = np.zeros((14, 14), dtype=object)
tabla_joc[6][6], tabla_joc[6][7], tabla_joc[7][6], tabla_joc[7][7] = '1', '2', '3', '4'
dictionar_piese = dict({
    (0, 1), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), 
    (8, 7), (9, 7), (10, 7), (11, 1), (12, 1), (13, 1), (14, 1), 
    (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), 
    (24, 1), (25, 1), (27, 1), (28, 1), (30, 1), (32, 1), (35, 1), 
    (36, 1), (40, 1), (42, 1), (45, 1), (48, 1), (49, 1), (50, 1), 
    (54, 1), (56, 1), (60, 1), (63, 1), (64, 1), (70, 1), (72, 1), 
    (80, 1), (81, 1), (90, 1)
})

for file in files:
    if file.endswith('.jpg'):
        print(file)
        if '_01' in file:
            img1 = cv.imread(img1_path)
            tabla_joc = np.zeros((14, 14), dtype=object)
            tabla_joc[6][6], tabla_joc[6][7], tabla_joc[7][6], tabla_joc[7][7] = '1', '2', '3', '4'

        img2_path = os.path.join(antrenare_dir, file)
        img2 = cv.imread(img2_path)

        result1 = extrage_careu(img1)
        result2 = extrage_careu(img2)

        res1_hsv = cv.cvtColor(result1, cv.COLOR_BGR2HSV)
        res2_hsv = cv.cvtColor(result2, cv.COLOR_BGR2HSV)

        l = np.array([0, 0, 0])
        u = np.array([85, 96, 255])
        mask_res1 = cv.inRange(res1_hsv, l, u)
        mask_res2 = cv.inRange(res2_hsv, l, u)

        kernel = np.ones((3, 3), np.uint8)
        mask_cleaned1 = cv.erode(mask_res1, kernel)
        mask_cleaned2 = cv.erode(mask_res2, kernel)

        
        # poz1, poz2 = determina_pozitie_piesa(mask_cleaned1, mask_cleaned2, lines_horizontal, lines_vertical)
        result = determina_pozitie_piesa(mask_cleaned1, mask_cleaned2, lines_horizontal, lines_vertical)
        if result is None:
            continue
        else:
            poz1, poz2 = result

        piese_posibile = determina_piese_posibile(tabla_joc, poz1, poz2, dictionar_piese)
        piesa = determina_piesa(result2, poz1, poz2, piese_posibile, lines_horizontal, lines_vertical)
        tabla_joc[poz1][poz2] = piesa
        dictionar_piese[piesa] -= 1

        alfabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']
        rezultat_path = os.path.join(rezultate_dir, f"{file[:-3]}txt")
        with open(rezultat_path, 'w') as fileWrite:
            fileWrite.write(f"{poz1 + 1}{alfabet[poz2]} {piesa}")

        img1 = cv.imread(img2_path)

1_01.jpg
1_02.jpg
1_03.jpg
1_04.jpg
1_05.jpg
1_06.jpg
1_07.jpg
1_08.jpg
1_09.jpg
1_10.jpg
1_11.jpg
1_12.jpg
1_13.jpg
1_14.jpg
1_15.jpg
1_16.jpg
1_17.jpg
1_18.jpg
1_19.jpg
1_20.jpg
1_21.jpg
1_22.jpg
1_23.jpg
1_24.jpg
1_25.jpg
1_26.jpg
1_27.jpg
1_28.jpg
1_29.jpg
1_30.jpg
1_31.jpg
1_32.jpg
1_33.jpg
1_34.jpg
1_35.jpg
1_36.jpg
1_37.jpg
1_38.jpg
1_39.jpg
1_40.jpg
1_41.jpg
1_42.jpg
1_43.jpg
1_44.jpg
1_45.jpg
1_46.jpg
1_47.jpg
1_48.jpg
1_49.jpg
1_50.jpg
2_01.jpg
2_02.jpg
2_03.jpg
2_04.jpg
2_05.jpg
2_06.jpg
2_07.jpg
2_08.jpg
2_09.jpg
2_10.jpg
2_11.jpg
2_12.jpg
2_13.jpg
2_14.jpg
2_15.jpg
2_16.jpg
2_17.jpg
2_18.jpg
2_19.jpg
2_20.jpg
2_21.jpg
2_22.jpg
2_23.jpg
2_24.jpg
2_25.jpg
2_26.jpg
2_27.jpg
2_28.jpg
2_29.jpg
2_30.jpg
2_31.jpg
2_32.jpg
2_33.jpg
2_34.jpg
2_35.jpg
2_36.jpg
2_37.jpg
2_38.jpg
2_39.jpg
2_40.jpg
2_41.jpg
2_42.jpg
2_43.jpg
2_44.jpg
2_45.jpg
2_46.jpg
2_47.jpg
2_48.jpg
2_49.jpg
2_50.jpg
3_01.jpg
3_02.jpg
3_03.jpg
3_04.jpg
3_05.jpg
3_06.jpg
3_07.jpg
3_08.jpg
3_09.jpg
3_10.jpg
3_11.jpg
3